In [1]:
import tensorflow as tf

2024-02-23 11:46:49.148040: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-23 11:46:49.262286: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 11:46:49.262386: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 11:46:49.262664: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 11:46:49.311094: I tensorflow/core/platform/cpu_feature_g

In [2]:
# # https://github.com/sonarsushant/California-House-Price-Prediction/blob/master/housing.csv
# dataset = tf.data.experimental.CsvDataset(
#     "./notebooks/california-housing.csv",
#     [
#         tf.constant(float("nan"), tf.float32),  # longitude
#         tf.constant(float("nan"), tf.float32),  # latitude
#         tf.constant(float("nan"), tf.float32),  # housing_median_age
#         tf.constant(float("nan"), tf.float32),  # total_rooms
#         tf.constant(float("nan"), tf.float32),  # total_bedrooms
#         tf.constant(float("nan"), tf.float32),  # population
#         tf.constant(float("nan"), tf.float32),  # households
#         tf.constant(float("nan"), tf.float32),  # median_income
#         tf.constant(float("nan"), tf.float32),  # median_house_value
#         tf.constant("", tf.string),  # ocean_proximity
#     ],
#     header=True,
# )
# dataset = dataset.filter(
#     lambda *all: tf.math.logical_not(tf.reduce_any(tf.math.is_nan(all[:9])))
# )
# dataset = dataset.map(
#     lambda longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value, ocean_proximity: (
#         longitude / 180.0,
#         latitude / 90.0,
#         housing_median_age / 100,
#         total_rooms / 10000,
#         total_bedrooms / 10000,
#         population/ 10000,
#         households/ 10000,
#         median_income/ 100,
#         median_house_value / 1000000,
#         ocean_proximity,
#     )
# )
# dataset = dataset.map(lambda *all: (tf.stack(all[:8]), tf.stack(all[8:9])))
# dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)

# for element in dataset.take(2):
#     print(element)

# for _ in dataset:
#     pass

In [3]:
# http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data
dataset = tf.data.experimental.CsvDataset(
    "./notebooks/auto-mpg.data.csv",
    [
        tf.constant(float("nan"), tf.float32),  # MPG
        tf.constant(float("nan"), tf.float32),  # Cylinders
        tf.constant(float("nan"), tf.float32),  # Displacement
        tf.constant(float("nan"), tf.float32),  # Horsepower
        tf.constant(float("nan"), tf.float32),  # Weight
        tf.constant(float("nan"), tf.float32),  # Acceleration
        tf.constant(float("nan"), tf.float32),  # Year
        tf.constant(0, tf.int32),  # Origin
        tf.constant("", tf.string),  # Name
    ],
    header=False,
    na_value="?"
)
dataset = dataset.filter(
    lambda *all: tf.math.logical_not(tf.reduce_any(tf.math.is_nan(all[:7])))
)
dataset = dataset.map(
    lambda MPG, Cylinders, Displacement, Horsepower, Weight, Acceleration, Year, Origin, Name: (
        MPG / 100,
        Cylinders / 10,
        Displacement / 1000,
        Horsepower / 1000,
        Weight / 10000,
        Acceleration / 100,
        Year / 100,
        Origin,
        Name,
    )
)
dataset = dataset.map(lambda *all: (tf.stack(all[1:7]), tf.stack(all[0:1])))
dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)

for element in dataset.take(2):
    print(element)

for _ in dataset:
    pass

2024-02-23 11:46:51.735312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-23 11:46:51.767175: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-23 11:46:51.767205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-23 11:46:51.780906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-23 11:46:51.780940: I tensorflow/compile

(<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0.8   , 0.307 , 0.13  , 0.3504, 0.12  , 0.7   ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.18], dtype=float32)>)
(<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0.8   , 0.35  , 0.165 , 0.3693, 0.115 , 0.7   ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.15], dtype=float32)>)


In [10]:
# self growing feed forward network

def self_growing_feed_forward_network(
    dataset, input_dimensions, output_dimensions, factor=0.9, max_hidden_layers=100
):
    inputs = tf.keras.layers.Input(shape=(input_dimensions,))
    hidden_layers = [inputs]
    output_layers = [
        tf.keras.layers.Dense(output_dimensions, kernel_initializer="zeros")(inputs)
    ]

    def train_while_loss_improves(factor=0.9, max_iterations=100):
        model = tf.keras.Model(
            inputs=inputs, outputs=tf.reduce_sum(output_layers, axis=0)
        )
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="mean_squared_error",
        )
        last_loss = float("inf")
        for iteration in range(max_iterations):
            print("TRAINING ITERATION " + str(iteration))
            history = model.fit(dataset, epochs=1)
            loss = history.history["loss"][-1]
            print(
                "TRANING LOSS IMPROVEMENT: "
                + str((loss / last_loss))
                + "| "
                + str(loss - last_loss)
            )
            if loss < last_loss * factor:
                last_loss = loss
            else:
                return model, loss
        return model, last_loss

    _, last_loss_ = train_while_loss_improves()
    for level in range(max_hidden_layers):
        print("TRANING LEVEL " + str(level))
        hidden_layer = tf.keras.layers.Dense(
            1, activation="relu", kernel_initializer="zeros"
        )(tf.concat(hidden_layers, axis=-1))
        output_layer = tf.keras.layers.Dense(
            output_dimensions, use_bias=False, kernel_initializer="ones"
        )(hidden_layer)
        hidden_layers.append(hidden_layer)
        output_layers.append(output_layer)
        model, loss = train_while_loss_improves()
        print(
            "TRANING LEVEL LOSS IMPROVEMENT: "
            + str((loss / last_loss_))
            + "| "
            + str(loss - last_loss_)
        )
        if loss < last_loss_ * factor:
            last_loss_ = loss
        else:
            return model

inp, out = next(iter(dataset))

basic_model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(inp.shape[-1],)),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(out.shape[-1]),
    ]
)
basic_model.compile(
    optimizer=tf.keras.optimizers.Adam(), loss="mean_squared_error"
)
basic_model.summary()
basic_model.fit(dataset.batch(32), epochs=100)

# model = self_growing_feed_forward_network(
#     dataset.batch(128),
#     input_dimensions=inp.shape[-1],
#     output_dimensions=out.shape[-1],
# )

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 64)                448       
                                                                 
 dense_26 (Dense)            (None, 64)                4160      
                                                                 
 dense_27 (Dense)            (None, 1)                 65        
                                                                 
Total params: 4673 (18.25 KB)
Trainable params: 4673 (18.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
25/25 [==============================] - 1s 4ms/step - loss: 0.0080
Epoch 2/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 0.0027
Epoch 4/50
25/25 [=============================

2024-02-23 11:51:03.455047: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12478095115415030079
2024-02-23 11:51:03.455122: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3486516967940654643
2024-02-23 11:51:03.455145: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9553506695798809560


2/2 [==============================] - 0s 7ms/step - loss: 8.8086e-04
Epoch 12/50
2/2 [==============================] - 0s 8ms/step - loss: 8.6721e-04
Epoch 13/50
2/2 [==============================] - 0s 5ms/step - loss: 8.9987e-04
Epoch 14/50
2/2 [==============================] - 0s 4ms/step - loss: 9.3794e-04
Epoch 15/50
2/2 [==============================] - 0s 5ms/step - loss: 9.4753e-04
Epoch 16/50
2/2 [==============================] - 0s 4ms/step - loss: 9.2645e-04
Epoch 17/50
2/2 [==============================] - 0s 6ms/step - loss: 8.9507e-04
Epoch 18/50
2/2 [==============================] - 0s 8ms/step - loss: 8.7326e-04
Epoch 19/50
2/2 [==============================] - 0s 12ms/step - loss: 8.6496e-04
Epoch 20/50
2/2 [==============================] - 0s 9ms/step - loss: 8.6271e-04
Epoch 21/50
2/2 [==============================] - 0s 5ms/step - loss: 8.5888e-04
Epoch 22/50
2/2 [==============================] - 0s 6ms/step - loss: 8.5293e-04
Epoch 23/50
2/2 [==========

In [7]:
def preview_model(model):
  i, o = next(iter(dataset.batch(1)))
  print((model.predict(i)[0][0], o[0][0].numpy()))

preview_model(basic_model)
preview_model(model)

1/1 [==============================] - 0s 58ms/step
(0.15505554, 0.18)
1/1 [==============================] - 0s 19ms/step
(0.21831633, 0.18)
